In [ ]:
!pip install afinn

     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=5f9ed50d6ceaf327a026c01333eadc2453114f3af7148a1c9381f5ace1b1c34d
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn


In [ ]:
import numpy as np 
import json 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from scipy import sparse 
import pickle 
from afinn import Afinn

In [ ]:
train_path ='./data/train_processed.csv'
val_path =  './data/val_processed.csv'
test_path = './data/test_processed.csv'

train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)

In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,user_id,business_id,stars,date,text,useful,...,year,month,day,language,cleaned_text,funny_label,cool_label,useful_label,funny_sampled_flag,cool_sampled_flag
0,0,0,0,O4RZMP8IFyJTNfRp0QXEsw,u0LXt3Uea_GidxRW1xcsfg,vcxvQyAggPqxcHwvJXvjGg,4,2017-01-04,Love this place!\n\nThe cakes are delicious bu...,0.0,...,2017.0,1.0,4.0,en,Love place cake delicious really rich cute cup...,0,0,0,0,0
1,1,1,1,66KqTwiQ1oB9-aTsoEN35Q,u0LXt3Uea_GidxRW1xcsfg,DKiRDPtQ5cTN-eX1oEgA9w,3,2017-01-04,It's a pub... nice and clean one.\n\nCame here...,0.0,...,2017.0,1.0,4.0,en,pub nice clean one Came grab food girl since T...,0,0,0,0,0
2,2,2,2,3KQ3_xutS5R3mX7HsPWpgw,u0LXt3Uea_GidxRW1xcsfg,zkU-WMio8g6dpRJ2Y2xqvQ,4,2017-01-04,Came here for dim sum - it's one of those mark...,0.0,...,2017.0,1.0,4.0,en,Came dim sum one mark sheet order dim sum pret...,0,0,0,0,0
3,3,3,3,p659sFGjbL8chCRZPPyFmg,u0LXt3Uea_GidxRW1xcsfg,3RlylOY452bA8rwliPUeUQ,4,2017-01-04,My sister loves this place! Probably comes by ...,1.0,...,2017.0,1.0,4.0,en,sister love place Probably come week every two...,0,0,1,1,1
4,4,4,4,1XEm85LhDeiUHdG0R_rXSQ,u0LXt3Uea_GidxRW1xcsfg,sTV4qyjDkY5cLvx2omtLyw,4,2017-02-07,Nice addition to the Distillery district! Stum...,1.0,...,2017.0,2.0,7.0,en,Nice addition Distillery district Stumbled upo...,0,0,1,0,0


In [ ]:
train['l1']=train.text.str.split().apply(len)
train['l2']=train.cleaned_text.str.split().apply(len)
train['l3']=train.text.apply(len)
train['l4']=train.cleaned_text.apply(len)

In [ ]:
afinn_emo = Afinn(emoticons=True)
afinn_scores = [afinn_emo.score(text) for text in train.text]
train['afinn'] = afinn_scores
train[['afinn', 'text']].head(10)

,afinn,text
0,16.0,Love this place!\n\nThe cakes are delicious bu...
1,18.0,It's a pub... nice and clean one.\n\nCame here...
2,2.0,Came here for dim sum - it's one of those mark...
3,10.0,My sister loves this place! Probably comes by ...
4,13.0,Nice addition to the Distillery district! Stum...
5,4.0,my sister is a big fan of the noodles here as ...
6,3.0,This is a most stop for me and my girlfriend w...
7,13.0,Saw that this was in my neighbourhood so decid...
8,15.0,Suzanne was able to get my business five T.V. ...
9,8.0,The saga has finished....because I needed to e...


In [ ]:
# train_use = train
train_fun = train[train.funny_sampled_flag==1]
train_cool = train[train.cool_sampled_flag==1]

In [ ]:
val['l1']=val.text.str.split().apply(len)
val['l2']=val.cleaned_text.str.split().apply(len)
val['l3']=val.text.apply(len)
val['l4']=val.cleaned_text.apply(len)

test['l1']=test.text.str.split().apply(len)
test['l2']=test.cleaned_text.str.split().apply(len)
test['l3']=test.text.apply(len)
test['l4']=test.cleaned_text.apply(len)

In [ ]:
afinn_emo = Afinn(emoticons=True)
afinn_scores = [afinn_emo.score(text) for text in val.text]
val['afinn'] = afinn_scores

afinn_emo = Afinn(emoticons=True)
afinn_scores = [afinn_emo.score(text) for text in test.text]
test['afinn'] = afinn_scores

In [ ]:
for i in [train, train_fun, train_cool, val, test]:
    i.dropna(inplace=True)
    print(i.shape)

(896945, 27)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(294305, 27)
(376157, 27)
(98119, 24)
(126864, 24)


In [ ]:
train.reset_index(inplace=True)
train_fun.reset_index(inplace=True)
train_cool.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
from sklearn import tree, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, mean_squared_error
from xgboost import XGBClassifier

# Useful

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/useful_processed_text/useful_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array, val_array, test_array]:
    print(i.shape)

(896945, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_new = np.zeros((train_array.shape[0],train_array.shape[1]+5))
train_array_new[:,:-5] = train_array
train_array_new[:,-5] = np.array(train.afinn.tolist())
train_array_new[:,-4] = np.array(train.l1.tolist())
train_array_new[:,-3] = np.array(train.l2.tolist())
train_array_new[:,-2] = np.array(train.l3.tolist())
train_array_new[:,-1] = np.array(train.l4.tolist())
train_array_new.shape

(896945, 65)

In [ ]:
val_array_new = np.zeros((val_array.shape[0],val_array.shape[1]+5))
val_array_new[:,:-5] = val_array
val_array_new[:,-5] = np.array(val.afinn.tolist())
val_array_new[:,-4] = np.array(val.l1.tolist())
val_array_new[:,-3] = np.array(val.l2.tolist())
val_array_new[:,-2] = np.array(val.l3.tolist())
val_array_new[:,-1] = np.array(val.l4.tolist())
val_array_new.shape

(98119, 65)

In [ ]:
test_array_new = np.zeros((test_array.shape[0],test_array.shape[1]+5))
test_array_new[:,:-5] = test_array
test_array_new[:,-5] = np.array(test.afinn.tolist())
test_array_new[:,-4] = np.array(test.l1.tolist())
test_array_new[:,-3] = np.array(test.l2.tolist())
test_array_new[:,-2] = np.array(test.l3.tolist())
test_array_new[:,-1] = np.array(test.l4.tolist())
test_array_new.shape

(126864, 65)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  1.0
confusion matrix: 
[[594845      0]
 [     0 302100]]
accuracy score:  0.6065797653869281
confusion matrix: 
[[49152 24449]
 [14153 10365]]
accuracy score:  0.6149813974019422
confusion matrix: 
[[66391 33639]
 [15206 11628]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6973415315320337
confusion matrix: 
[[537629  57216]
 [214252  87848]]
accuracy score:  0.7351889032705184
confusion matrix: 
[[64866  8735]
 [17248  7270]]
accuracy score:  0.7540831126245429
confusion matrix: 
[[87049 12981]
 [18217  8617]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(min_samples_split=130)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.7619664527925346
confusion matrix: 
[[524214  70631]
 [142872 159228]]
accuracy score:  0.6699110264067102
confusion matrix: 
[[56628 16973]
 [15415  9103]]
accuracy score:  0.6829675873376214
confusion matrix: 
[[76222 23808]
 [16412 10422]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10,min_samples_split=130)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6965800578630796
confusion matrix: 
[[537219  57626]
 [214525  87575]]
accuracy score:  0.7349952608567148
confusion matrix: 
[[64841  8760]
 [17242  7276]]
accuracy score:  0.7540673477109345
confusion matrix: 
[[87049 12981]
 [18219  8615]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=200)
dt_estimator.fit(train_array_new, train.useful_label)
y_train_useful = dt_estimator.predict(train_array_new)
y_val_useful = dt_estimator.predict(val_array_new)
y_test_useful = dt_estimator.predict(test_array_new)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.690077986944573
confusion matrix: 
[[541101  53744]
 [224239  77861]]
accuracy score:  0.74276134082084
confusion matrix: 
[[66002  7599]
 [17641  6877]]
accuracy score:  0.7629429940723925
confusion matrix: 
[[88675 11355]
 [18719  8115]]


# Funny

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/funny_processed_text/funny_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_fun, val_array_fun, test_array_fun]:
    print(i.shape)

(294305, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_fun_new = np.zeros((train_array_fun.shape[0],train_array_fun.shape[1]+5))
train_array_fun_new[:,:-5] = train_array_fun
train_array_fun_new[:,-5] = np.array(train_fun.afinn.tolist())
train_array_fun_new[:,-4] = np.array(train_fun.l1.tolist())
train_array_fun_new[:,-3] = np.array(train_fun.l2.tolist())
train_array_fun_new[:,-2] = np.array(train_fun.l3.tolist())
train_array_fun_new[:,-1] = np.array(train_fun.l4.tolist())
train_array_fun_new.shape

(294305, 65)

In [ ]:
val_array_fun_new = np.zeros((val_array_fun.shape[0],val_array_fun.shape[1]+5))
val_array_fun_new[:,:-5] = val_array_fun
val_array_fun_new[:,-5] = np.array(val.afinn.tolist())
val_array_fun_new[:,-4] = np.array(val.l1.tolist())
val_array_fun_new[:,-3] = np.array(val.l2.tolist())
val_array_fun_new[:,-2] = np.array(val.l3.tolist())
val_array_fun_new[:,-1] = np.array(val.l4.tolist())
val_array_fun_new.shape

(98119, 65)

In [ ]:
test_array_fun_new = np.zeros((test_array_fun.shape[0],test_array_fun.shape[1]+5))
test_array_fun_new[:,:-5] = test_array_fun
test_array_fun_new[:,-5] = np.array(test.afinn.tolist())
test_array_fun_new[:,-4] = np.array(test.l1.tolist())
test_array_fun_new[:,-3] = np.array(test.l2.tolist())
test_array_fun_new[:,-2] = np.array(test.l3.tolist())
test_array_fun_new[:,-1] = np.array(test.l4.tolist())
test_array_fun_new.shape

(126864, 65)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  1.0
confusion matrix: 
[[176583      0]
 [     0 117722]]
accuracy score:  0.61749508250186
confusion matrix: 
[[56221 32935]
 [ 4596  4367]]
accuracy score:  0.6175510783200908
confusion matrix: 
[[73228 43208]
 [ 5311  5117]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.668908105536773
confusion matrix: 
[[145406  31177]
 [ 66265  51457]]
accuracy score:  0.7651830939981044
confusion matrix: 
[[71099 18057]
 [ 4983  3980]]
accuracy score:  0.7616345062429057
confusion matrix: 
[[91773 24663]
 [ 5577  4851]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=15)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.7289954299111466
confusion matrix: 
[[146243  30340]
 [ 49418  68304]]
accuracy score:  0.7124410155015848
confusion matrix: 
[[65438 23718]
 [ 4497  4466]]
accuracy score:  0.7070327279606508
confusion matrix: 
[[84347 32089]
 [ 5078  5350]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_fun_new, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun_new)
y_val_funny = dt_estimator.predict(val_array_fun_new)
y_test_funny = dt_estimator.predict(test_array_fun_new)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6563327160598699
confusion matrix: 
[[144692  31891]
 [ 69252  48470]]
accuracy score:  0.7753951833997493
confusion matrix: 
[[72215 16941]
 [ 5097  3866]]
accuracy score:  0.7740336107958128
confusion matrix: 
[[93433 23003]
 [ 5664  4764]]


# Cool

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/cool_processed_text/cool_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_cool, val_array_cool, test_array_cool]:
    print(i.shape)

(376157, 60)
(98119, 60)
(126864, 60)


In [ ]:
train_array_cool_new = np.zeros((train_array_cool.shape[0],train_array_cool.shape[1]+5))
train_array_cool_new[:,:-5] = train_array_cool
train_array_cool_new[:,-5] = np.array(train_cool.afinn.tolist())
train_array_cool_new[:,-4] = np.array(train_cool.l1.tolist())
train_array_cool_new[:,-3] = np.array(train_cool.l2.tolist())
train_array_cool_new[:,-2] = np.array(train_cool.l3.tolist())
train_array_cool_new[:,-1] = np.array(train_cool.l4.tolist())
train_array_cool_new.shape

(376157, 65)

In [ ]:
val_array_cool_new = np.zeros((val_array_cool.shape[0],val_array_cool.shape[1]+5))
val_array_cool_new[:,:-5] = val_array_cool
val_array_cool_new[:,-5] = np.array(val.afinn.tolist())
val_array_cool_new[:,-4] = np.array(val.l1.tolist())
val_array_cool_new[:,-3] = np.array(val.l2.tolist())
val_array_cool_new[:,-2] = np.array(val.l3.tolist())
val_array_cool_new[:,-1] = np.array(val.l4.tolist())
val_array_cool_new.shape

(98119, 65)

In [ ]:
test_array_cool_new = np.zeros((test_array_cool.shape[0],test_array_cool.shape[1]+5))
test_array_cool_new[:,:-5] = test_array_cool
test_array_cool_new[:,-5] = np.array(test.afinn.tolist())
test_array_cool_new[:,-4] = np.array(test.l1.tolist())
test_array_cool_new[:,-3] = np.array(test.l2.tolist())
test_array_cool_new[:,-2] = np.array(test.l3.tolist())
test_array_cool_new[:,-1] = np.array(test.l4.tolist())
test_array_cool_new.shape

(126864, 65)

In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  1.0
confusion matrix: 
[[225694      0]
 [     0 150463]]
accuracy score:  0.5992417370743689
confusion matrix: 
[[52369 31930]
 [ 7392  6428]]
accuracy score:  0.6030000630596545
confusion matrix: 
[[68420 41975]
 [ 8390  8079]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=20)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.7820112346706295
confusion matrix: 
[[203770  21924]
 [ 60074  90389]]
accuracy score:  0.7050316452470979
confusion matrix: 
[[63509 20790]
 [ 8152  5668]]
accuracy score:  0.709578761508387
confusion matrix: 
[[82600 27795]
 [ 9049  7420]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=30)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.9274664568252087
confusion matrix: 
[[217211   8483]
 [ 18801 131662]]
accuracy score:  0.6345967651525188
confusion matrix: 
[[56021 28278]
 [ 7575  6245]]
accuracy score:  0.639992432841468
confusion matrix: 
[[73262 37133]
 [ 8539  7930]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_cool_new, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool_new)
y_val_cool = dt_estimator.predict(val_array_cool_new)
y_test_cool = dt_estimator.predict(test_array_cool_new)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6552157742644693
confusion matrix: 
[[192467  33227]
 [ 96466  53997]]
accuracy score:  0.7780042601331036
confusion matrix: 
[[71138 13161]
 [ 8621  5199]]
accuracy score:  0.783729032664901
confusion matrix: 
[[92502 17893]
 [ 9544  6925]]
